<a href="https://colab.research.google.com/github/nitrogenlab/GP15_watermassanalysis/blob/main/GP02OCIMandOMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook first uses fluxes from a pre-defined OCIM (Ocean Circulation Inverse Model) to explain the composition of each gridbox in the ocean corresponding to the GP02 cruise transect in terms of the fractional contributions from user-defined `end-members'. These results are then used to refine the solution produced by pyompa (a software for conducting OMP analysis) to choose an OMP solution that most resembles the OCIM solution while maintaining residuals comparable to the best OMP solution.

This notebook was adapted from a notebook of Avanti Shrikumar (avanti@cs.stanford.edu)'s notebooks for GP02 by Rian Lawrence in the Casciotti lab at Stanford (https://nitrogen.stanford.edu/)

# Use OCIM to perform water mass analysis for the GP02 transect

Our goal will be to explain observations gathered from the GP02 cruise in terms of user-specified end-members using the fluxes from OCIM2.

## Install and import needed software


### Install miniconda

resources used--
https://saturncloud.io/blog/conda-environment-in-google-colab-a-guide-for-software-engineers/ and https://saturncloud.io/blog/how-to-install-conda-package-to-google-colab/ and https://www.linkedin.com/pulse/how-install-run-conda-google-colab-ambu-vijayan/

setting up

In [1]:
%env PYTHONPATH=
#CLEARED PYTHON PATH

env: PYTHONPATH=


In [2]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2023-08-28 22:46:50--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M   162MB/s    in 0.5s    

2023-08-28 22:46:50 (162 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - crypto

Don't forget to reload the browser page after you installed the kernel (as explained in the original post). https://stackoverflow.com/questions/60775160/install-python-3-8-kernel-in-google-colaboratory/71511943#71511943

CHECK VERSION OF PYTHON

In [3]:
!which python
!python --version
!echo $PYTHONPATH

/usr/local/bin/python
Python 3.8.17



In [57]:
#! wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
#! chmod +x Miniconda3-latest-Linux-x86_64.sh
#! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local/
#!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
#!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local

#above doesn't work because scikits.umfpack does not work with later versions
#of python and conda

--2023-08-27 20:35:37--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103219356 (98M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh.2’

Miniconda3-latest-L 100%[===================>]  98.44M   148MB/s    in 0.7s    

2023-08-27 20:35:38 (148 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh.2’ saved [103219356/103219356]

PREFIX=/usr/local
Unpacking payload ...
                                                                                   
Installing base environment...



Preparing transaction: - done
Executing transaction: | done
installation finished.


In [17]:
! conda install -c conda-forge scikit-umfpack --yes
import sys
sys.path.append("/usr/local/lib/python3.11/site-packages")


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/requests/compat.py", line 11, in <module>
        import chardet
    ModuleNotFoundError: No module named 'chardet'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/conda/exception_handler.py", line 16, in __call__
        return func(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/main.py", line 84, in main_subshell
        exit_code = do_call(args, p)
                    ^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/conda_argparse.py", line 124, in do_call
        module = import_module(relative_mod, __name__.rsplit(".", 1)[0])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "

In [9]:
import scikits.umfpack as umfpack

ModuleNotFoundError: ignored

check to get version number. if no version number somethign went wrong

In [58]:
!conda --version


conda 23.5.2


In [59]:
!which conda

/usr/local/bin/conda


# Appending to the sys.path

In [49]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [12]:
!ls /usr/local/lib/python3.10/dist-packages

absl
absl_py-1.4.0.dist-info
aiohttp
aiohttp-3.8.5.dist-info
aiosignal
aiosignal-1.3.1.dist-info
alabaster
alabaster-0.7.13.dist-info
albumentations
albumentations-1.3.1.dist-info
altair
altair-4.2.2.dist-info
annotated_types
annotated_types-0.5.0.dist-info
anyio
anyio-3.7.1.dist-info
apiclient
appdirs-1.4.4.dist-info
appdirs.py
apt
apt_inst.cpython-310-x86_64-linux-gnu.so
apt_inst-stubs
apt_pkg.cpython-310-x86_64-linux-gnu.so
apt_pkg-stubs
aptsources
argon2
argon2_cffi-23.1.0.dist-info
_argon2_cffi_bindings
argon2_cffi_bindings-21.2.0.dist-info
array_record
array_record-0.4.1.dist-info
arviz
arviz-0.15.1.dist-info
astropy
astropy-5.3.2.dist-info
astunparse
astunparse-1.6.3.dist-info
async_timeout
async_timeout-4.0.3.dist-info
attr
attrs
attrs-23.1.0.dist-info
audioread
audioread-3.0.0.dist-info
AUTHORS.rst
autograd
autograd-1.6.2.dist-info
babel
Babel-2.12.1.dist-info
backcall
backcall-0.2.0.dist-info
beautifulsoup4-4.11.2.dist-info
bin
bleach
bleach-6.0.0.dist-info
blis
blis-0.7.10.d

In [55]:
import sys
sys.path.append("/usr/local/lib/python3.10/site-packages")

### Install packages

In [71]:
!pip install -q condacolab
import condacolab
condacolab.install()

ModuleNotFoundError: ignored

Install scikit-umfpack for faster solutions of sparse linear systems

In [48]:
! conda install scikit-umfpack --yes
#! conda install -q -y --prefix /usr/local python=3.10 scikit-umfpack


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/requests/compat.py", line 11, in <module>
        import chardet
    ModuleNotFoundError: No module named 'chardet'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/conda/exception_handler.py", line 16, in __call__
        return func(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/main.py", line 84, in main_subshell
        exit_code = do_call(args, p)
                    ^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/conda_argparse.py", line 124, in do_call
        module = import_module(relative_mod, __name__.rsplit(".", 1)[0])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "

Install METIS for better row-ordering during matrix factorization (greatly reduces memory usage)

In [53]:
! conda install metis --yes
#! conda install -q -y --prefix /usr/local python=3.10 metis


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/requests/compat.py", line 11, in <module>
        import chardet
    ModuleNotFoundError: No module named 'chardet'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/conda/exception_handler.py", line 16, in __call__
        return func(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/main.py", line 84, in main_subshell
        exit_code = do_call(args, p)
                    ^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/conda_argparse.py", line 124, in do_call
        module = import_module(relative_mod, __name__.rsplit(".", 1)[0])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "

In [52]:
! conda install -c conda-forge netcdf4 --yes
#! conda install -q -y --prefix /usr/local python=3.7 netcdf4


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/requests/compat.py", line 11, in <module>
        import chardet
    ModuleNotFoundError: No module named 'chardet'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/usr/local/lib/python3.11/site-packages/conda/exception_handler.py", line 16, in __call__
        return func(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/main.py", line 84, in main_subshell
        exit_code = do_call(args, p)
                    ^^^^^^^^^^^^^^^^
      File "/usr/local/lib/python3.11/site-packages/conda/cli/conda_argparse.py", line 124, in do_call
        module = import_module(relative_mod, __name__.rsplit(".", 1)[0])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "

In [31]:
#! conda install -c conda-forge gsw
#! conda install -q -y --prefix /usr/local python=3.7 gsw

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - gsw

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




Install pyompa for conducting OMP analysis (pyompa was developed by the Casciotti lab and supports several capabilities beyond traditional OMP analysis)

In [1]:
!pip uninstall -y pyompa
%cd /content/
!rm -rf pyompa
!git clone https://github.com/nitrogenlab/pyompa
%cd /content/pyompa
!git checkout dev #install from the dev branch
!git log -1
!pip install .
%cd /content/

/content
Cloning into 'pyompa'...
remote: Enumerating objects: 1408, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1408 (delta 160), reused 94 (delta 94), pack-reused 1244
Receiving objects: 100% (1408/1408), 16.71 MiB | 10.07 MiB/s, done.
Resolving deltas: 100% (999/999), done.
/content/pyompa
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
commit e1490ff156ed8db745c73abf8daa9ea4aa3ac3b5 (HEAD -> dev, origin/dev)
Author: Av Shrikumar <avanti.shrikumar@gmail.com>
Date:   Mon Feb 28 08:58:09 2022 -0700

    updated with total parameter residual plots
Processing /content/pyompa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

/content


Install and load the gp15wma module, which has configuration settings for the GP15 OMP analysis as well as other handy functions for loading the data

In [2]:
!pip uninstall -y GP02wma
%cd /content/
!rm -rf gp15wmascripts
!git clone https://github.com/nitrogenlab/gp15wmascripts
%cd /content/gp15wmascripts
!git log -1
!pip install .
%cd /content/

/content
Cloning into 'gp15wmascripts'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 316 (delta 53), reused 3 (delta 3), pack-reused 226
Receiving objects: 100% (316/316), 16.58 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (177/177), done.
/content/gp15wmascripts
commit 1cb48fba3bebcf2234e6bad60ff58880d6563493 (HEAD -> main, origin/main, origin/HEAD)
Author: rml54 <45828236+rml54@users.noreply.github.com>
Date:   Sun Aug 13 17:25:37 2023 -0400

    Update funcdump.py
Processing /content/gp15wmascripts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.7 MB/s eta 0:00:00
  Created wheel for gp15wma: filename=gp15wma-0.1.0.0-py3-none-any.whl size=15116 sha256=fce822f87eba06e8b41348179c44a780fc3f4d1c835206229bbbd8ec42989fe3
  Stored in directory: /tmp/pip-ephem-wheel-cache-71k8vtie/wheels/76/d6/19/c421f224b7b298b98f2fc8c5bbc3cfb00344

### Import all the relevant modules

In [ ]:
!pip uninstall -y pyompa
%cd /content/
!rm -rf pyompa
!git clone https://github.com/nitrogenlab/pyompa
%cd /content/pyompa
!git checkout dev #install from the dev branch
!git log -1
!pip install .
%cd /content/

In [7]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 18.5 MB/s eta 0:00:00


In [18]:
!pip install metis

  Preparing metadata (setup.py) ... done
  Created wheel for metis: filename=metis-0.2a5-py3-none-any.whl size=11305 sha256=58fd6c823f2780433c6736a414ca1dc6c1986a4d982054a7e139146b1d33292c
  Stored in directory: /root/.cache/pip/wheels/20/0c/3a/2fe1d1d85fd5d04cb86520271fa12e529b77ba83737e4738fa
Successfully built metis


In [4]:
!pip install https://github.com/nitrogenlab/GP15_watermassanalysis/blob/main/scikit_learn-0.24.2-cp310-cp310-win32.whl


ERROR: scikit_learn-0.24.2-cp310-cp310-win32.whl is not a supported wheel on this platform.


In [3]:
!pip install scikit-umfpack

! wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

  Using cached scikit-umfpack-0.3.3.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-umfpack
  Running setup.py clean for scikit-umfpack
Failed to build scikit-umfpack
ERROR: Could not build wheels for scikit-umfpack, which is required to install pyproject.toml-based projects


In [10]:
import numpy as np
from matplotlib import pyplot as plt
#import gsw
import pandas
from collections import OrderedDict, namedtuple
from datetime import datetime
import json
import gc
import scipy.io
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import (isspmatrix_csc, isspmatrix_csr, isspmatrix,
                          SparseEfficiencyWarning, csc_matrix, csr_matrix)
from scipy.sparse.sputils import is_pydata_spmatrix
import scikits.umfpack as umfpack
import copy
import netCDF4
import pyompa
import GP02wma
from GP02wma import gsw

<ipython-input-10-b73cac3d6059>:15: DeprecationWarning: Please use `is_pydata_spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import is_pydata_spmatrix


ModuleNotFoundError: ignored

## Load the OCIM data

### Download the OCIM data

OCIM data are from https://figshare.com/articles/dataset/OCIM2-48L_base_state_model_output/14802732

In [ ]:
#Only try to download if the tgz file doesn't exist
![[ -e OCIM2_48L_base.tar.gz ]] || wget https://figshare.com/ndownloader/files/28468077 -O OCIM2_48L_base.tar.gz
#Unzip the files
!tar -xzf OCIM2_48L_base.tar.gz

### Load the transport matrix

We load the original transport matrix

In [ ]:
base_transport = scipy.io.loadmat("OCIM2_48L_base_transport.mat")['TR']

### Load other relevant base data

Load latitude, longitude and depth co-ordinates for gridboxes from OCIM2_48L_base_data.nc.

In [ ]:
data = netCDF4.Dataset("OCIM2_48L_base_data.nc")
#ocnmask is a mask indicating whether a grid cell is in the ocean
# the ocean grid cells correspond to the entries in the transport matrix
ocnmask = data.variables['ocnmask'][:].data==1.0
#ulat, ulon and wz are the lat/lon/depth coordinates corresponding to the fluxes
ulat = data.variables['ulon'][:].data #latlon are swapped in the nc file
ulon = data.variables['ulat'][:].data #latlon are swapped in the nc file
wz = data.variables['wz'][:].data

#get the mapping from depth/lon/lan to idx within ocnvec
depth_idxs, lon_idxs, lat_idxs = np.indices(ocnmask.shape)
ocnvec_mapping = np.zeros_like(ocnmask).astype("int")
ocnvec_mapping[:] = np.iinfo(np.int).max
ocnvec_mapping[(depth_idxs.ravel()[ocnmask.ravel()],
                lon_idxs.ravel()[ocnmask.ravel()],
                lat_idxs.ravel()[ocnmask.ravel()])] = np.arange(np.sum(ocnmask))
del depth_idxs, lon_idxs, lat_idxs

As a sanity check, let's do a scatterplot of all the lat/lon coordinates corresponding to the ocean gridpoints. We note that the OCIM longitude co-ordinates go fro 0 to 360.

In [ ]:
plt.scatter(ulon[ocnmask], ulat[ocnmask])
plt.xlabel("Longitude (OCIM co-ordinates)")
plt.ylabel("Latitude")
plt.show()

Get the latitude, longitude and depth specifically at ocean gridboxes. Also convert longitudes to have negative values west of the prime meridian, consistent with more standard notation.

In [ ]:
#the _ocnvec suffix indicates the vector of properties corresponds to grid
# boxes that are in the ocean; it lines up with the axes of the transport matrix
ulat_ocnvec = ulat.ravel()[ocnmask.ravel()]
ulon_ocnvec = ulon.ravel()[ocnmask.ravel()]
#convert ulon to have negative longitudes to the west of the prime meridian,
# which makes it more compatible with what other packages expect
ulon_ocnvec = (ulon_ocnvec*(ulon_ocnvec <= 180)
               + (-(360-ulon_ocnvec)*(ulon_ocnvec > 180)))

depth_ocnvec = wz.ravel()[ocnmask.ravel()]

As a sanity check, we'll make sure that the converted longitude coordinates correspond to what we want (i.e. negative values west of the prime meridian)

In [ ]:
plt.scatter(ulon_ocnvec, ulat_ocnvec)
plt.xlabel("Longitude (standard co-ordinates)")
plt.ylabel("Latitude")
plt.show()

## Load the World Ocean Atlas data

### Download the WOA data

Key for reading the WOA data:
- s=salinity, t=temperature, i=silicate, o=oxygen  
- _an is the objectively analyzed value   
- decav is the decadal average  
- A5B7 is 2005 to 2017  (A=2000s, B=2010s). "All" indicates all time ranges. "all" is only available for nutrients and is not available for temp/salinity
- The 00 in s00/t00/i00/o00 indicates it's an annual value  
- 1.00 indicates it's for a 1-degree-resolution grid, as does the 01 suffix in the file name

In [ ]:
#WOA18 for A5B7 salinity
! [[ -e woa18_A5B7_s00_01.nc ]] || wget https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/salinity/netcdf/A5B7/1.00/woa18_A5B7_s00_01.nc -O woa18_A5B7_s00_01.nc

#WOA18 A5B7 temperature
! [[ -e woa18_A5B7_t00_01.nc ]] || wget https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/temperature/netcdf/A5B7/1.00/woa18_A5B7_t00_01.nc -O woa18_A5B7_t00_01.nc

#WOA18 "all" silicate
! [[ -e woa18_all_i00_01.nc ]] || wget https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/silicate/netcdf/all/1.00/woa18_all_i00_01.nc -O woa18_all_i00_01.nc

#WOA18 "all" oxygen
! [[ -e woa18_all_o00_01.nc ]] || wget https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/oxygen/netcdf/all/1.00/woa18_all_o00_01.nc -O woa18_all_o00_01.nc


### Load WOA into a pandas data frame

In [ ]:
def extract_woa_into_df(attr_to_woancdata):
    attributes = list(attr_to_woancdata.keys())
    lat_ticks = attr_to_woancdata[attributes[0]]['lat'][:].data
    lon_ticks = attr_to_woancdata[attributes[0]]['lon'][:].data
    depth_ticks = attr_to_woancdata[attributes[0]]['depth'][:].data
    #make sure the ticks are the same across all woancdata entries, as a
    # sanity check
    for attr in attr_to_woancdata:
        assert tuple(lat_ticks) == tuple(attr_to_woancdata[attr]['lat'][:].data)
        assert tuple(lon_ticks) == tuple(attr_to_woancdata[attr]['lon'][:].data)
        assert tuple(depth_ticks) == tuple(
                                       attr_to_woancdata[attr]['depth'][:].data)

    attr_to_data = dict([(attr, attr_to_woancdata[attr][attr][:].data)
                         for attr in attr_to_woancdata])
    attr_to_nanfill = dict([(attr, attr_to_woancdata[attr][attr]._FillValue)
                          for attr in attr_to_woancdata])

    pandas_dict = OrderedDict([
        ('depth', []),
        ('latitude', []),
        ('longitude', [])
    ])
    for attr in attr_to_woancdata:
        pandas_dict[attr] = []

    for depthidx in range(len(depth_ticks)):
        for latidx in range(len(lat_ticks)):
            for lonidx in range(len(lon_ticks)):
                #only consider points where data is present
                # for some of the readings
                data_present = any([(
                    attr_to_data[attr][0, depthidx, latidx, lonidx]
                    != attr_to_nanfill[attr]) for attr in attributes])
                if (data_present):
                    pandas_dict['depth'].append(depth_ticks[depthidx])
                    pandas_dict['latitude'].append(lat_ticks[latidx])
                    pandas_dict['longitude'].append(lon_ticks[lonidx])
                    for attr in attr_to_data:
                        dataval =\
                            attr_to_data[attr][0, depthidx, latidx, lonidx]
                        if (dataval == attr_to_nanfill[attr]):
                            dataval = np.nan
                        pandas_dict[attr].append(dataval)
    return pandas.DataFrame(pandas_dict)

woa18_df = extract_woa_into_df(
              attr_to_woancdata={
                  't_an': netCDF4.Dataset("woa18_A5B7_t00_01.nc"),
                  's_an': netCDF4.Dataset("woa18_A5B7_s00_01.nc"),
                  'i_an': netCDF4.Dataset("woa18_all_i00_01.nc"),
                  'o_an': netCDF4.Dataset("woa18_all_o00_01.nc")
                  })

Add conservative temperature, absolute salinity and potential density to the WOA data frame

In [ ]:
def augment_woa_with_conservative_vals(woa_df):
    depth = np.array(woa_df["depth"])
    lats = np.array(woa_df["latitude"])
    lons = np.array(woa_df["longitude"])
    #pressure calculation from depth
    p = gsw.p_from_z(
      z=-depth, #z is expected to be negative in the ocean
      lat=lats)
    #get the absolute salinity given the practical salinity and pressure
    abssal = gsw.SA_from_SP(SP=np.array(woa_df["s_an"]), #practical salinity
                            p=p, lon=lons, lat=lats)
    #get conservative temp given absolute salinity, temp and pressure
    ctemp = gsw.CT_from_t(SA=abssal,
                          t=np.array(woa_df["t_an"]), #temperature
                          p=p)
    #get sigma2/sigma2/sigma4, which will be used for defining the locations of
    # endmembers
    sig0 = gsw.sigma0(SA=abssal, CT=ctemp)
    sig2 = gsw.sigma2(SA=abssal, CT=ctemp)
    sig4 = gsw.sigma4(SA=abssal, CT=ctemp)

    woa_df["pressure"] = p
    woa_df["absolute_salinity"] = abssal
    woa_df["conservative_temperature"] = ctemp
    woa_df["sigma0"] = sig0
    woa_df["sigma2"] = sig2
    woa_df["sigma4"] = sig4

augment_woa_with_conservative_vals(woa_df=woa18_df)

### Index WOA data by OCIM gridboxes

We organize the measurements into a data structure such that we can easily retrieve the measurements corresponding to particular OCIM gridboxes

First, we get the depth, latitude and longitude 'axes tickmarks' corresponding to the OCIM gridboxes

In [ ]:
ocim_wz_ticks = tuple(wz[:,0,0])
ocim_ulat_ticks = tuple(ulat[0,0,:])
ocim_ulon_ticks = tuple(ulon[0,:,0])

#also delete obsolete variables
del data
del ulat
del ulon
del wz
gc.collect()

Next, we define functions that can be used for mapping the measured data into a data structure corresponding to the OCIM gridboxes. Each observation is assigned to the OCIM gridbox that has the closest value of ulat/ulon/depth.

In [ ]:
#This function initializes an empty grid
def initialize_grid(shape):
    grid = []
    for depth_idx in range(shape[0]):
        grid_depthentry = []
        for lon_idx in range(shape[1]):
            grid_lonentry = []
            for lat_idx in range(shape[2]):
                grid_latentry = []
                grid_lonentry.append(grid_latentry)
            grid_depthentry.append(grid_lonentry)
        grid.append(grid_depthentry)
    return grid

#This function maps observations in a data frame into a grid
def prepare_obsgrid_from_df(df, wz_ticks, ulon_ticks, ulat_ticks,
                                depth_key, longitude_key, latitude_key,
                                keys_to_record, return_idx_mapping=False):
    obs_grid = initialize_grid((len(wz_ticks), len(ulon_ticks), len(ulat_ticks)))

    ObsEntry = namedtuple('ObsEntry', keys_to_record)

    df_lats = np.array(df[latitude_key])
    df_lons = np.array(df[longitude_key])
    df_depths = np.array(df[depth_key])

    #np.searchsorted gives the insertion index in order
    # to maintain sort order
    #Also need to convert the lons to the 0-360 system used by OCIM
    df_lons_converted = (df_lons*(df_lons >= 0.0)
                         + (df_lons + 360)*(df_lons < 0.0))
    lat_insert_idxs = np.searchsorted(a=ulat_ticks, v=df_lats)
    lon_insert_idxs = np.searchsorted(a=ulon_ticks, v=df_lons_converted)
    depth_insert_idxs = np.searchsorted(a=wz_ticks, v=df_depths)

    #also get the contents associated with keys_to_record, to avoid
    # storing unnecessarily large amounts of data
    key_to_coldata = dict([
        (key, np.array(df[key])) for key in keys_to_record
    ])

    if (return_idx_mapping):
        mapped_idxs = [[], [], []] #depth, lon, lat

    num_obs_skipped = 0
    for i in range(len(lat_insert_idxs)):
        lat_insert_idx = lat_insert_idxs[i]
        lon_insert_idx = lon_insert_idxs[i]
        depth_insert_idx = depth_insert_idxs[i]

        item_lat = df_lats[i]
        item_lon = df_lons_converted[i]
        item_depth = df_depths[i]

        #In an Arakawa-B grid, the velocities are measured in the center of
        # the grid. Thus, we want to map the observation to the gridbox
        # for which the lat/lon/depth measurement is closer.
        lat_idx = (lat_insert_idx
              if (abs(ulat_ticks[lat_insert_idx] - item_lat)
                  < abs(ulat_ticks[lat_insert_idx-1] - item_lat))
              else lat_insert_idx-1)
        #Need to do the %360 to account for wrap-around
        lon_idx = (lon_insert_idx
                  if (abs(ulon_ticks[lon_insert_idx] - item_lon)%360
                      < abs(ulon_ticks[lon_insert_idx-1] - item_lon)%360)
                   else lon_insert_idx-1)
        #If the insertion index is deeper than the tick of the deepest gridbox,
        # the index is the one corresponding to the deepest gridbox (0-indexed)
        if (depth_insert_idx == len(wz_ticks)):
            depth_idx = depth_insert_idx - 1
        else:
            depth_idx = (depth_insert_idx
            if ((abs(wz_ticks[depth_insert_idx] - item_depth)
                < abs(wz_ticks[depth_insert_idx-1] - item_depth))
                ##avoid going too deep if that gridbox is on land
                #and (ocnvec_mapping[depth_insert_idx,lon_idx,lat_idx]
                #     != np.iinfo(np.int).max)
                )
            else depth_insert_idx-1)

        #In case the gridbox mapped to corresponds to a land box, then go
        # higher at look at adjacent lat/lon until we find a non-land
        # gridbox
        c_depthidx = depth_idx
        c_lonidx = lon_idx
        c_latidx = lat_idx

        #If we don't have to return the index mapping, then we can
        # ignore cases where something maps to a land gridbox
        if (return_idx_mapping == False):
          if (ocnmask[c_depthidx, c_lonidx, c_latidx]==False):
              num_obs_skipped += 1
              continue
        else: #otherwise, go through the process of finding a match
            while (ocnmask[c_depthidx, c_lonidx, c_latidx]==False):
                if ocnmask[c_depthidx-1, c_lonidx, c_latidx]:
                    c_depthidx = c_depthidx-1 #look higher first
                    break
                #cycle through lat/lon surroundings
                if ocnmask[c_depthidx, c_lonidx-1, c_latidx]:
                    c_lonidx = c_lonidx-1
                    break
                if ocnmask[c_depthidx, (c_lonidx+1) % ocnmask.shape[1], c_latidx]:
                    c_lonidx = c_lonidx+1
                    break
                if ocnmask[c_depthidx, c_lonidx, c_latidx-1]:
                    c_latidx = c_latidx-1
                    break
                if ocnmask[c_depthidx, c_lonidx, (c_latidx+1)  % ocnmask.shape[2]]:
                    c_latidx = c_latidx+1
                    break
                if (c_depthidx==0):
                    break #don't keep looking if reached surface
                #go higher and do search again
                c_depthidx = c_depthidx - 1

        if ( (c_depthidx, c_lonidx, c_latidx) != (depth_idx, lon_idx, lat_idx)):
            print("Warning!",(item_depth, item_lon, item_lat), "is closest"
              +" to land gridbox",
              (wz_ticks[depth_idx], ulon_ticks[lon_idx],ulat_ticks[lat_idx]),
              "so mapping to",
              (wz_ticks[c_depthidx], ulon_ticks[c_lonidx],ulat_ticks[c_latidx]),
              "gridbox instead")
            depth_idx, lon_idx, lat_idx = (c_depthidx, c_lonidx, c_latidx)
        if (return_idx_mapping):
            #if (ocnvec_mapping[depth_idx,lon_idx,lat_idx]) == np.iinfo(np.int).max:
            #    print(depth_idx, lon_idx, lat_idx)
            #    print(item_depth, item_lon, item_lat)
            #    print(ocnvec_mapping[depth_idx,lon_idx,lat_idx],
            #          ocnvec_mapping[depth_idx-1,lon_idx,lat_idx],
            #          ocnvec_mapping[depth_idx-2,lon_idx,lat_idx],
            #          ocnvec_mapping[depth_idx-3,lon_idx,lat_idx])
            mapped_idxs[0].append(depth_idx)
            mapped_idxs[1].append(lon_idx)
            mapped_idxs[2].append(lat_idx)
        obs_grid[depth_idx][lon_idx][lat_idx].append(
            ObsEntry(*[key_to_coldata[key][i] for key in keys_to_record ] ))

    if (return_idx_mapping == False):
      print("Skipped",num_obs_skipped,"observations out of",
            len(lat_insert_idxs),"as the closest gridbox was a land gridbox")

    if (return_idx_mapping):
        return obs_grid, mapped_idxs
    else:
        return obs_grid

Now we run those functions to map the WOA data into gridboxes

In [ ]:
woa18_obs_grid = prepare_obsgrid_from_df(df=woa18_df,
    wz_ticks=ocim_wz_ticks, ulon_ticks=ocim_ulon_ticks,
    ulat_ticks=ocim_ulat_ticks,
    depth_key="depth", longitude_key="longitude", latitude_key="latitude",
    keys_to_record=["conservative_temperature", "absolute_salinity",
                    "i_an", "o_an", "sigma0", "sigma2", "sigma4"])

### Compute average WOA data in OCIM gridboxes

Compute things like the average silicate concentration, average conservative temperature of the WOA data in each OCIM gridbox.

In [ ]:
#Function that averages the observations in each gridbox,
# returning np.nan when there are no observations in a gridbox
def get_data_from_obs_grid(func, obs_grid):
    to_return = np.zeros(
        (len(obs_grid), len(obs_grid[0]),
         len(obs_grid[0][0])) )
    to_return[:,:,:] = np.nan
    for i in range(len(obs_grid)):
        for j in range(len(obs_grid[i])):
            for k in range(len(obs_grid[i][j])):
                if len(obs_grid[i][j][k]) > 0:
                    to_return[i,j,k] = func(obs_grid[i][j][k])
    return to_return

def get_mean_from_obs_grid(attr, obs_grid):
    return get_data_from_obs_grid(
              func=eval("lambda arr: np.mean([y."+attr+" for y in arr])"),
              obs_grid=obs_grid)

#Get the mean conservative temperature, absolute salinity,
# and silicate in the ocean gridboxes with WOA18 data
woa18_mean_ctemp_ocnvec = get_mean_from_obs_grid(
    attr="conservative_temperature",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]
woa18_mean_abssal_ocnvec = get_mean_from_obs_grid(
    attr="absolute_salinity",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]
woa18_mean_silicate_ocnvec = get_mean_from_obs_grid(
    attr="i_an",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]

#also get the potential density and oxygen data as this will be
# useful for defining end-member boundaries
woa18_mean_sig0_ocnvec = get_mean_from_obs_grid(
    attr="sigma0",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]
woa18_mean_sig2_ocnvec = get_mean_from_obs_grid(
    attr="sigma2",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]
woa18_mean_sig4_ocnvec = get_mean_from_obs_grid(
    attr="sigma4",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]
woa18_mean_oxygen_ocnvec = get_mean_from_obs_grid(
    attr="o_an",
    obs_grid=woa18_obs_grid).ravel()[ocnmask.ravel()]

Save memory by deleting the objects corresponding to `woa18_obs_grid` and `woa18_df`

In [ ]:
del woa18_obs_grid
del woa18_df

gc.collect()

## Idenitify OCIM gridboxes corresponding to end-members

We define endmembers by lat/lon/potential density and figure out the indexes in the vector of ocean gridboxes that correspond to each endmember. We also use WOA data to add in a filter on oxygen.

In [ ]:
#Function to get the indexes that match a lat/lon/potential density threshold
# range. These are indexes into the vector of ocean gridboxes.
def get_endmember_idxs(lat_min, lat_max,
                      lon_min, lon_max,
                      invert_lon=False,
                      depth_min=0, depth_max=np.inf,
                      sig0_min=0, sig0_max=np.inf,
                      sig2_min=0, sig2_max=np.inf,
                      sig4_min=0, sig4_max=np.inf,
                      ox_min=0, ox_max=np.inf):
    nonzero_idxs = np.nonzero(
      (ulat_ocnvec >= lat_min)*(ulat_ocnvec <= lat_max)
     *((((ulon_ocnvec) >= lon_min)*((ulon_ocnvec) <= lon_max))
       ==(False if invert_lon else True))
     *(depth_ocnvec >= depth_min)*(depth_ocnvec <= depth_max)
     *(woa18_mean_sig0_ocnvec >= sig0_min)*(woa18_mean_sig0_ocnvec <= sig0_max)
     *(woa18_mean_sig2_ocnvec >= sig2_min)*(woa18_mean_sig2_ocnvec <= sig2_max)
     *(woa18_mean_sig4_ocnvec >= sig4_min)*(woa18_mean_sig4_ocnvec <= sig4_max)
     *(woa18_mean_oxygen_ocnvec >= ox_min)
     *(woa18_mean_oxygen_ocnvec <= ox_max)
    )[0]
    return nonzero_idxs

endmember_definitions = {
    "AAIW": {"lat_min":-55.0, "lat_max":-43.0,
             "lon_min":-90, "lon_max":-80,
             "sig0_min":27.05, "sig0_max":27.15},
    "NPIW": {"lat_min":36.5, "lat_max":39.0,
             "lon_min":148.0, "lon_max":154.0,
             "sig0_min":26.4, "sig0_max":26.9},
    "UCDW": {"lat_min":-49.5, "lat_max":-44.5,
             "lon_min":-157.0, "lon_max":-147.0,
             "sig0_min":27.35, "sig0_max":27.75},
    "LCDW": {"lat_min":-66.5, "lat_max":-61.5,
             "lon_min":-100.0, "lon_max":150.0,
             "invert_lon":True,
             "sig0_min":27.79, "sig0_max":27.83},
    "AABW": {"lat_min":-66.5, "lat_max":-61.5,
             "lon_min":-100.0, "lon_max":150.0,
             "invert_lon":True,
             "sig4_min":46.04, "sig4_max":200,
             "depth_min":1500},
    "PDW": {"lat_min":39.0, "lat_max":51.0,
             "lon_min":-170.0, "lon_max":-133.0,
             "sig0_min":27.2, "sig0_max":200,
             "sig4_min":0, "sig4_max":45.88},
    "EqIW": {"lat_min":-5, "lat_max":5,
             "lon_min":-90.0, "lon_max":-80.0,
             "sig0_min": 26.86, "sig0_max": 27.3},
    "PSUW": {"lat_min":50, "lat_max":58,
             "lon_min":-155, "lon_max":-140,
             "sig0_min": 25.29, "sig0_max": 26.5},
    "ENPCW": {"lat_min":16, "lat_max":26,
             "lon_min":-170, "lon_max":-140,
             "sig0_min": 25.29, "sig0_max": 26.5},
    "ESSW": {"lat_min":-5, "lat_max":5,
             "lon_min":-90.0, "lon_max":-80.0,
             "sig0_min": 25.29, "sig0_max": 26.86},
    "SPCW": {"lat_min":-30, "lat_max":-20,
             "lon_min":-152, "lon_max":-130,
             "sig0_min": 25.29, "sig0_max": 26.86},
}

endmem_to_idxs = dict([(the_key, set(get_endmember_idxs(**val)))
                        for the_key,val in endmember_definitions.items()])

Delete variables that are no-longer needed

In [ ]:
del woa18_mean_sig0_ocnvec
del woa18_mean_sig2_ocnvec
del woa18_mean_sig4_ocnvec
del woa18_mean_oxygen_ocnvec

gc.collect()

As a sanity check, we can visualize the locations on the map for each endmember

In [ ]:
endmem_names = sorted(endmem_to_idxs.keys())

for endmem in endmem_names:
    print(endmem)
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20,5))

    plt.sca(ax[0])
    #first make a scatterplot of the water surface
    plt.scatter(ulon_ocnvec,
            ulat_ocnvec, color="blue")
    #now overlay the endmember
    #handles.append(
    plt.scatter(
        ulon_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        ulat_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        color="red")
    #)

    plt.sca(ax[1])
    plt.scatter(
        ulat_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        depth_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        color="red")
    plt.xlim(-90,90)
    plt.ylim(max(depth_ocnvec),0)

    plt.sca(ax[2])
    plt.scatter(
        ulon_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        depth_ocnvec[np.array(list(endmem_to_idxs[endmem]))],
        color="red")
    plt.xlim(-180,180)
    plt.ylim(max(depth_ocnvec),0)
    plt.show()


## Load the GP02 data

We load the GP02 data so that we can compare how the predictions from OCIM along the GP02 transect compare to the empirical measurements on the GP02 cruise. We focus on intermediate and deep waters here.

In [ ]:
GP02_df, GP02_intermediateanddeep, GP02_thermocline = GP02wma.download_and_load_GP02_data(
    station_to_tc_cutoffs_url="https://github.com/nitrogenlab/GP15_watermassanalysis/blob/0a64ed0faca01701cf6c84d09365abc706594e2c/GP02_station_to_tc_cutoffs.json")
del GP02_df, GP02_thermocline

### Index GP02 data by OCIM gridboxes

We can reuse the code from when we read the GLODAP data into OCIM gridboxes

In [ ]:
gp15_obs_grid, gp15_obsgrid_idxmapping =\
  prepare_obsgrid_from_df(
    df=GP02_intermediateanddeep,
    wz_ticks=ocim_wz_ticks, ulon_ticks=ocim_ulon_ticks,
    ulat_ticks=ocim_ulat_ticks,
    depth_key="Depth", longitude_key="lon", latitude_key="lat",
    keys_to_record=["conservative_temp", "absolute_salinity", "silicate"],
    return_idx_mapping=True)

### Compute average GP02 data in OCIM gridboxes

As before, we reuse the code we wrote for doing this on GLODAP data

In [ ]:
#Get the mean conservative temperature, absolute salinity and silicate
gp15_mean_ctemp_ocnvec = get_mean_from_obs_grid(
    attr="conservative_temp",
    obs_grid=gp15_obs_grid).ravel()[ocnmask.ravel()]
gp15_mean_asbsal_ocnvec = get_mean_from_obs_grid(
    attr="absolute_salinity",
    obs_grid=gp15_obs_grid).ravel()[ocnmask.ravel()]
gp15_mean_silicate_ocnvec = get_mean_from_obs_grid(
    attr="silicate",
    obs_grid=gp15_obs_grid).ravel()[ocnmask.ravel()]

Delete objects to save memory

In [ ]:
del gp15_obs_grid
gc.collect()

## Compute the distributions at steady-state

We compute the distributions of tracers and end-members at steady state, treating the end-members as sources for the rest of the ocean

### Prepare code to compute steady-state distributions

There are some bugs associated with Google Colab's version of scipy in terms of how it interfaces with umfpack. To fix this, I have ported over an alternative versions of the relevant functions from https://github.com/scipy/scipy/pull/11453.

I have also modified the functions to use the METIS ordering for the LU decomposition, since the default umfpack ordering runs out of memory without this.

In [ ]:
#see https://github.com/scipy/scipy/pull/11453

def _get_umf_family(A):
    """Get umfpack family string given the sparse matrix dtype."""
    _families = {
        (np.float64, np.int32): 'di',
        (np.complex128, np.int32): 'zi',
        (np.float64, np.int64): 'dl',
        (np.complex128, np.int64): 'zl'
    }

    f_type = np.sctypeDict[A.dtype.name]
    i_type = np.sctypeDict[A.indices.dtype.name]

    try:
        family = _families[(f_type, i_type)]

    except KeyError:
        msg = 'only float64 or complex128 matrices with int32 or int64' \
            ' indices are supported! (got: matrix: %s, indices: %s)' \
            % (f_type, i_type)
        raise ValueError(msg)

    # See gh-8278. Considered converting only if
    # A.shape[0]*A.shape[1] > np.iinfo(np.int32).max,
    # but that didn't always fix the issue.
    family = family[0] + "l"
    A_new = copy.copy(A)
    A_new.indptr = np.array(A.indptr, copy=False, dtype=np.int64)
    A_new.indices = np.array(A.indices, copy=False, dtype=np.int64)

    return family, A_new


def get_umf_context(A):
    if is_pydata_spmatrix(A):
        A = A.to_scipy_sparse().tocsc()

    if not isspmatrix_csc(A):
        A = csc_matrix(A)
        warn('splu requires CSC matrix format', SparseEfficiencyWarning)

    A = A.asfptype()  # upcast to a floating point format

    if A.dtype.char not in 'dD':
        raise ValueError("convert matrix data to double, please, using"
              " .astype(), or set linsolve.useUmfpack = False")

    umf_family, A = _get_umf_family(A)
    umf = umfpack.UmfpackContext(umf_family)
    #METIS ordering ends up taking up MUCH less memory!
    umf.control[umfpack.UMFPACK_ORDERING] = umfpack.UMFPACK_ORDERING_METIS

    #see 'alternative routines' section of the UMFPACK user guide
    # page 17 describes the orderings
    #Also see documentation for umfpackcontext:
    # https://github.com/scikit-umfpack/scikit-umfpack/blob/a2102ef92f4dd060138e72bb5d7c444f8ec49cbc/scikits/umfpack/umfpack.py#L114

    return umf, A


#creating a fixed version of factorize
def fixed_factorize(A):
    umf, A = get_umf_context(A)
    # Make LU decomposition.
    umf.numeric(A)
    def solve(b):
        return umf.solve(umfpack.UMFPACK_A, A, b, autoTranspose=True)
    return solve


#create a fixed version of solve
def fixed_solve(A, b):
    umf, A = get_umf_context(A)

    return umf.linsolve(umfpack.UMFPACK_A, A, b,
                         autoTranspose=True)

    return solve


Now we are ready to define the function to get the steady-state tracer concentrations by solving the linear system.

In [ ]:

class GetSteadyStateTracerConcsViaMatrixSolve(object):

    """
    T: the 2d transport matrix
    source_idxs: the set of indexes to designate as source gridboxes
    """
    def __init__(self, T, source_idxs, tau=None):
        print(datetime.now(), "Prepping solver")
        self.tau = tau
        to_factorize = self.get_tofactorize(T=T, source_idxs=source_idxs)
        #self.find_conditionnumber(mat=to_factorize)
        #Don't invert the matrix!
        # https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/
        self.solver = fixed_factorize(to_factorize)
        print(datetime.now(), "solver prepped")

    @staticmethod
    def find_conditionnumber(mat):
        print(datetime.now(),"Computing A A^t")
        AAt = mat @ mat.transpose()
        print(datetime.now(),"Getting largest eigenvalue")
        largest_eig = scipy.sparse.linalg.eigsh(A=AAt, k=1, which='LM',
                                                return_eigenvectors=False)
        print(datetime.now(), largest_eig)
        print(datetime.now(),"Getting smallest eigenvalue")
        smallest_eig = scipy.sparse.linalg.eigsh(A=AAt, k=1, which='SM',
                                                 sigma=0,
                                                 return_eigenvectors=False)
        print(datetime.now(), smallest_eig)
        print("Ratio:", largest_eig/smallest_eig)

    #source_vecs should be set to the empirically-determined concentration
    # of the tracer for the source gridboxes, and should be 0 for all
    # non-source gridboxes.
    def __call__(self, source_vecs):
        print(datetime.now(), "Calling solver")
        if (self.tau is not None):
            return self.solver(-source_vec/self.tau)
        else:
            return self.solver(-source_vec)
        print(datetime.now(), "Solver called")

    def get_tofactorize(self, T, source_idxs):
        #Return the matrix that needs to be factorized
        # [(I - M)T - M] for the tau=None case
        # (T - M/tau) for the other case
        m_vec = np.zeros(T.shape[0])
        m_vec[source_idxs] = 1.0
        M = scipy.sparse.diags(m_vec)
        if (self.tau is not None):
            return csc_matrix(T - M/self.tau)
        else:
            I_minus_M = scipy.sparse.diags(1 - m_vec)
            return csc_matrix(I_minus_M@T - M)



### Instantiate the solver

Instantiate a solver that's appropriate for the source indices we are using in this case

In [ ]:
source_idxs = np.array(list(set(np.concatenate(
                  [list(x) for x in endmem_to_idxs.values()], axis=0))))

solver = GetSteadyStateTracerConcsViaMatrixSolve(
            T=base_transport,
            source_idxs=source_idxs,
            #tau=None,
            tau=(1.0/(60*60*24*365))
            )

### Compute the steady-state tracer distributions

With the solver instantiated, we can compute the expected tracer concentrations based on the OCIM fluxes, assuming that the sources for the tracers are the defined end-members

In [ ]:
tracers = [
    ("absolute salinity", woa18_mean_abssal_ocnvec, gp15_mean_asbsal_ocnvec),
    ("conservative temp", woa18_mean_ctemp_ocnvec, gp15_mean_ctemp_ocnvec),
    ("silicate", woa18_mean_silicate_ocnvec, gp15_mean_silicate_ocnvec)
]

gp15_idxs = ocnvec_mapping[tuple(gp15_obsgrid_idxmapping)]
#gp15_idxs = np.nonzero((np.isnan(gp15_mean_ctemp_ocnvec)==False))

fig, ax = plt.subplots(nrows=1, ncols=len(tracers),
                       figsize=(len(tracers)*7, 5))

tracer_to_ocim_residuals = {}

for i,(tracer_name, woa18_tracer_vals, gp15_tracer_vals) in enumerate(tracers):
    print("On ",tracer_name)
    #source vec has to be zero at all entries that are not sources. At
    # source entries, it should equal the empirically measured values
    source_vec = np.zeros(base_transport.shape[0])
    source_vec[source_idxs] = woa18_tracer_vals[source_idxs]
    steady_state_soln = solver(source_vec)

    plt.sca(ax[i])
    #Compute the residual with respect to the GP15 observation
    residuals = (steady_state_soln[gp15_idxs] - gp15_tracer_vals[gp15_idxs])
    tracer_to_ocim_residuals[tracer_name] = residuals
    plt.scatter(
        ulat_ocnvec[gp15_idxs],
        depth_ocnvec[gp15_idxs],
        c=residuals,
        vmin=-np.max(np.abs(residuals)), vmax=np.max(np.abs(residuals)),
        cmap="RdBu"
    )
    plt.ylim(plt.ylim()[1], plt.ylim()[0])
    plt.ylabel("Depth")
    plt.xlabel("Latitude")
    plt.title(tracer_name+" OCIM residuals\n(compared to GP15 measurement)")
    plt.colorbar()

plt.show()

### Compute the steady-state end-member distributions

Eeach end-member is like a tracer that has a value of 1 at all source gridboxes occupied by the end-member, and a value of 0 at all source gridboxes that are not occupied by the end-member. We will also define an end-member called "_ALL" that is the union of all endmembers in order to diagnose whether we may have missing end-members (if there are no end-members missing, then the end-member fractions should sum up close to 1 everywhere)

We will plot each end-member both on a fixed 0-1 scale (bottom) and on a relative scale that is adjusted to the minimum/maximum concentration of that end-member. The latter allows us to see the end-member distribution more clearly, while the former allows us to compare results for different end-members.

In [ ]:
#find steady-state distributions

endmem_to_idxs["_ALL"] = source_idxs

fig, ax = plt.subplots(nrows=2, ncols=len(endmem_to_idxs),
                       figsize=(len(endmem_to_idxs)*7, 10))

endmemname_to_ocimfracs = {}

for i, endmem in enumerate(sorted(endmem_to_idxs.keys())):
    print("On:", endmem)
    endmem_idxs = np.array(list(endmem_to_idxs[endmem]))
    source_vec = np.zeros(base_transport.shape[0])
    source_vec[endmem_idxs] = 1.0
    steady_state_soln = solver(source_vec)

    endmemname_to_ocimfracs[endmem] = steady_state_soln[gp15_idxs]

    print("Globally:")
    print("max val:",np.max(steady_state_soln))
    print("min val:",np.min(steady_state_soln))
    print("Along GP02:")
    print("max val:",np.max(steady_state_soln[gp15_idxs]))
    print("min val:",np.min(steady_state_soln[gp15_idxs]))
    #If the transport matrix has no off-diagonal negative entries, then
    # numerical errors can lead to frac values less than 0 or more than 1; get
    # rid of this
    #steady_state_soln = np.maximum(0, np.minimum(steady_state_soln, 1.0))

    plt.sca(ax[0,i])
    plt.scatter(
        ulat_ocnvec[gp15_idxs],
        depth_ocnvec[gp15_idxs],
        c=steady_state_soln[gp15_idxs]
    )
    plt.ylim(plt.ylim()[1], plt.ylim()[0])
    plt.ylabel("Depth")
    plt.xlabel("Latitude")
    plt.title(endmem+" OCIM fractions")
    plt.colorbar()

    plt.sca(ax[1,i])
    plt.scatter(
        ulat_ocnvec[gp15_idxs],
        depth_ocnvec[gp15_idxs],
        c=steady_state_soln[gp15_idxs],
        vmin=0.0, vmax=1.0
    )
    plt.ylim(plt.ylim()[1], plt.ylim()[0])
    plt.ylabel("Depth")
    plt.xlabel("Latitude")
    plt.title(endmem+" OCIM fractions (0-1 scale)")
    plt.colorbar()

plt.show()

#save endmemname_to_ocimfracs for use in other notebooks
np.save("endmemname_to_ocimfracs", endmemname_to_ocimfracs)

# Perform OMP analysis for the GP02 transect

We will use the pyompa implementation of OMP, as this has several additional features/improvements over traditional OMP analysis.

## Download and load pyompa end-member definitions

We focus on intermediate and deep waters here. We will use the gp02wma package.

In [ ]:
interanddeep_endmember_df = GP02wma.load_interanddeep_endmember_df(
    df_url="https://raw.githubusercontent.com/nitrogenlab/GP15_watermassanalysis/main/GP15_intermediateanddeep_endmemberswithsubtypes.csv",
    df_file_name="GP15_intermediateanddeep_endmemberswithsubtypes.csv")

## Run pyompa

We use the settings in the GP02wma package

In [ ]:
pyompa_soln = GP02wma.get_pyompa_soln(
    obs_df=gp15_intermediateanddeep,
    endmember_df_touse=interanddeep_endmember_df)

## Plot pyompa base solution

Plot the residuals

In [ ]:
from pyompa import plot_ompasoln_residuals
plot_ompasoln_residuals(pyompa_soln, xaxis_colname="lat", yaxis_colname="Depth")

Plot the endmember fractions in the "base" solution (this is just one solution, without the uncertainty analysis)

In [ ]:
from pyompa import plot_ompasoln_endmember_fractions
plot_ompasoln_endmember_fractions(pyompa_soln, xaxis_colname="lat",
                                  yaxis_colname="Depth")

# Use OCIM to refine the OMPA results

Our pyompa system is underdetermined, i.e. the solution is ambiguous. We can use OCIM to select a specific solution that achieves similar residuals to the best pyompa solution and also matches the OCIM solution as closely as possible

In [ ]:
!pip uninstall -y pyompa
%cd /content/
!rm -rf pyompa
!git clone https://github.com/nitrogenlab/pyompa
%cd /content/pyompa
!git checkout devsmallchanges
!git log -1
!pip install .
%cd /content/

%reload_ext autoreload
%autoreload 2
from importlib import reload
import pyompa
reload(pyompa.endmemberpenaltyfunc)
reload(pyompa)

In [ ]:
#set up the matrices needed for enforcing the penalties
obj_weights = np.zeros((
    len(pyompa_soln.endmembername_to_indices),
    len(pyompa_soln.endmember_names) + pyompa_soln.ompa_problem.num_converted_variables))
target_endmem_fracs = np.zeros((len(gp15_idxs),
                                len(pyompa_soln.endmembername_to_indices)))

for endmemberoverallidx, (endmembername, endmember_idxs) in enumerate(pyompa_soln.endmembername_to_indices.items()):
    obj_weights[endmemberoverallidx, endmember_idxs] = 1.0
    target_endmem_fracs[:, endmemberoverallidx] =\
      endmemname_to_ocimfracs[endmembername]

#compute the solution - setting resids to 0 constrains them to be no worse
# than what was obtained in the optimal pyompa soln
max_resids = 0.0*np.array([0.001, 0.004, 0.4, 0.1, 0.015, 0.2])
ocim_pyompa_soln = pyompa.ompacore.OMPASoln.core_quantify_ambiguity_via_residual_limits(
    self=pyompa_soln, obj_weights=obj_weights,
    max_resids=max_resids, retain_original_penalties=True,
    target_endmem_fracs=target_endmem_fracs,
    verbose=True, max_iter=1000000)

In [ ]:
from pyompa import plot_ompasoln_residuals, plot_ompasoln_endmember_fractions
plot_ompasoln_residuals(ocim_pyompa_soln, xaxis_colname="lat", yaxis_colname="Depth")
plot_ompasoln_endmember_fractions(ocim_pyompa_soln, xaxis_colname="lat", yaxis_colname="Depth")